In [1]:
import os
import math
import json
import itertools
import pandas as pd
import collections
import concurrent.futures as cf

In [3]:
blog_path = "../blogs/"
path_list = os.listdir(path=blog_path)
BLOG_SAMPLE_SIZE = 40
MAX_SEEN_VALUE = .50
MIN_SEEN_VALUE = .01
MATCH_LENIENCY = .50

In [4]:
def strip_blogs(sample_size, blog_path, paths):
    blogs = []
    for ind in range(sample_size):
        file = open(blog_path+paths[ind], "rb")
        posts = [line.decode("utf-8", errors="ignore").strip() for line in file]
        posts = [line for line in posts if (line != "" and line[0] != "<")]
        blogs.append(" ".join(posts))
        file.close()
    return blogs

In [5]:
# splits up a string into a list of words.
# str -> [str]
def get_words(str):
    words = []
    str = str+"."
    last_ind = 0
    for ind in range(len(str)):
        if not (str[ind].isalpha()):
            if last_ind == ind:
                last_ind += 1
                continue
            else:
                words.append(str[last_ind:ind].lower())
                last_ind = ind+1
    return words

In [6]:
def build_people_and_find_words(blogs):
    people = [dict(collections.Counter(blog)) for blog in blogs]
    flattened_blogs = [word for blog in blogs for word in blog]
    all_words_seen = dict(collections.Counter(flattened_blogs))
    return remove_useless_words(people, all_words_seen)

In [7]:
def remove_useless_words(people, all_words_seen):
    total_people = len(people)
    words_to_remove = [word for word in all_words_seen if ((all_words_seen[word]/total_people > MAX_SEEN_VALUE) or (all_words_seen[word]/total_people < MIN_SEEN_VALUE))]
    for word in words_to_remove:
        del all_words_seen[word]
        for person in people:
                if word in person:
                    del person[word]

    for word in [term for term in all_words_seen if term not in words_to_remove]:
        for person in [d for d in people if word not in d ]:
            person[word] = 0
    return (people, all_words_seen)

In [8]:
def term_frequency(person):
    return .5 + (.5*person/person.max())

In [9]:
# cos_dist = sum(ser1*ser2)/(sqrt(sum(ser1^2)) * sqrt(sum(ser2^2)))
def cos_dist(ser1, ser2, tdm, square_sums):
    numerator = tdm[ser1].dot(tdm[ser2])
    ser1_denominator = square_sums[ser1]
    ser2_denominator = square_sums[ser2]
    return 1 - numerator/(ser1_denominator*ser2_denominator)

In [10]:
def find_sums_for_each_person(tdm):
    square_sums = {person:math.sqrt(sum(map((lambda x: x**2), tdm[person]))) for person in tdm}
    return square_sums

def find_IDFs(blogs):
    blogs = list(map(set, blogs))
    blogs = list(map(collections.Counter, blogs))
    c = collections.Counter()
    for blog in blogs:
        c += blog
    return {word:math.log(BLOG_SAMPLE_SIZE/c[word]) for word in c}

In [11]:
def build_tdm(people, IDF_Dict, all_words_seen):
    tdm = pd.DataFrame({path_list[ind]:people[ind] for ind in range(BLOG_SAMPLE_SIZE) }, dtype=float)

    # (This step scales really badly)
    # IDF_Dict = {}
    # for word in all_words_seen:
    #     personCount = len([person for person in tdm if tdm[person][word] > 0])
    #     if personCount > 0:
    #         IDF_Dict[word] = math.log(BLOG_SAMPLE_SIZE / personCount)
    

    # for person in tdm:
    #     tdm[person] = term_frequency(tdm[person])
    #     for i in range(len(person)):
    #         tdm[person][i] = tdm[person][i] * IDF_Dict[tdm[person].keys()[i]]
    return tdm

In [ ]:
def parse_occupation(path_name):
    return path_name.split(".")[3]

def find_unique_occupations(path_list):
    uniques = set([parse_occupation(path) for path in path_list])
    return dict(zip(uniques, range(len(uniques))))

def dict_add_person(nodes_and_links, occupations, person):
        occupation = occupations[person.name.split(".")[3]]
        nodes_and_links["nodes"].append({"id": person.name, "group":occupation})
        for relation in person.keys():
            if (person[relation] < MATCH_LENIENCY) and (relation != person.name) : 
                nodes_and_links["links"].append({"source":person.name, "target":relation, "value":1})
                
def output_to_json(write_path, path_list, similarity_frame):
    nodes_and_links = {}
    nodes_and_links["nodes"] = []
    nodes_and_links["links"] = []

    occupations = find_unique_occupations(path_list)

    for person in similarity_frame:
        dict_add_person(nodes_and_links, occupations, similarity_frame[person])

    file = open(write_path, "w")
    file.write(json.dumps(nodes_and_links, sort_keys=True, indent=2))
    file.close()

In [14]:

blogs = strip_blogs(BLOG_SAMPLE_SIZE, blog_path, path_list)
pool = cf.ProcessPoolExecutor()
blogs = list(pool.map(get_words, blogs))
(people, all_words_seen) = build_people_and_find_words(blogs)
IDFs = find_IDFs(blogs)
tdm = build_tdm(people, IDFs, all_words_seen)
# (This step scales really badly)
square_sums = find_sums_for_each_person(tdm)
similarity_frame = pd.DataFrame({person:{relation:cos_dist(person, relation, tdm, square_sums) for relation in tdm} for person in tdm})
#     print(similarity_frame)
#     output_to_json("./writeTest.json", path_list, similarity_frame)

In [42]:
(similarity_frame < .5).all()

1000331.female.37.indUnk.Leo.xml  \
1000331.female.37.indUnk.Leo.xml                                        0.000000   
1004904.male.23.Arts.Capricorn.xml                                      1.000000   
1005076.female.25.Arts.Cancer.xml                                       0.997064   
1005545.male.25.Engineering.Sagittarius.xml                             0.982813   
1007188.male.48.Religion.Libra.xml                                      0.996844   
100812.female.26.Architecture.Aries.xml                                 0.976605   
1008329.female.16.Student.Pisces.xml                                    0.991998   
1009572.male.25.indUnk.Cancer.xml                                       0.994609   
1011153.female.27.Technology.Virgo.xml                                  0.994358   
1011289.female.25.indUnk.Libra.xml                                      0.973900   
1011311.female.17.indUnk.Scorpio.xml                                    0.979069   
1013637.male.17.RealEstate.Virgo.xml                                    0.986963   
1015252.female.23.indUnk.Pisces.xml                                     0.975914   
1015556.male.34.Technology.Virgo.xml                                    0.968773   
1016560.male.41.Publishing.Sagittarius.xml                              0.993296   
1016738.male.26.Publishing.Libra.xml                                    0.975149   
1016787.female.24.Communications-Media.Leo.xml                          1.000000   
1019224.female.27.RealEstate.Libra.xml                                  0.992850   
1019622.female.24.indUnk.Aquarius.xml                                   0.989156   
1019710.male.16.Student.Pisces.xml                                      0.996182   
1021779.female.25.indUnk.Scorpio.xml                                    0.976574   
1022037.male.23.indUnk.Cancer.xml                                       0.989487   
1022086.female.17.Student.Cancer.xml                                    0.980660   
1024234.female.17.indUnk.Libra.xml                                      0.997792   
1025783.female.17.Student.Gemini.xml                                    0.984778   
1026164.female.23.Education.Aries.xml                                   0.964479   
1026443.female.15.Student.Scorpio.xml                                   0.982010   
1028027.female.16.indUnk.Libra.xml                                      0.988073   
1028257.male.26.Education.Aries.xml                                     1.000000   
1029959.male.17.indUnk.Aries.xml                                        0.985591   
1031806.male.17.Technology.Sagittarius.xml                              0.983435   
1032153.male.27.Technology.Pisces.xml                                   0.975932   
1032591.female.24.Banking.Aquarius.xml                                  0.993079   
1032824.female.15.Student.Libra.xml                                     0.980395   
1034874.female.43.Publishing.Capricorn.xml                              0.988119   
1039136.male.24.Student.Capricorn.xml                                   0.989050   
1039908.female.16.indUnk.Gemini.xml                                     0.991731   
1040084.male.17.indUnk.Taurus.xml                                       0.981125   
1042993.male.15.Student.Sagittarius.xml                                 0.986116   
1043329.male.23.Government.Pisces.xml                                   0.991078   

                                                1004904.male.23.Arts.Capricorn.xml  \
1000331.female.37.indUnk.Leo.xml                                      1.000000e+00   
1004904.male.23.Arts.Capricorn.xml                                   -2.220446e-16   
1005076.female.25.Arts.Cancer.xml                                     9.688447e-01   
1005545.male.25.Engineering.Sagittarius.xml                           9.778086e-01   
1007188.male.48.Religion.Libra.xml                                    9.962799e-01   
100812.female.26.Architecture.Aries.xml                               9.816130e-01   
1

In [53]:
less_than = similarity_frame < .5 
greater_than = similarity_frame >.00000001

In [56]:
table = less_than.eq(greater_than)
table.any()

1000331.female.37.indUnk.Leo.xml                  False
1004904.male.23.Arts.Capricorn.xml                False
1005076.female.25.Arts.Cancer.xml                 False
1005545.male.25.Engineering.Sagittarius.xml       False
1007188.male.48.Religion.Libra.xml                False
100812.female.26.Architecture.Aries.xml           False
1008329.female.16.Student.Pisces.xml              False
1009572.male.25.indUnk.Cancer.xml                 False
1011153.female.27.Technology.Virgo.xml            False
1011289.female.25.indUnk.Libra.xml                False
1011311.female.17.indUnk.Scorpio.xml              False
1013637.male.17.RealEstate.Virgo.xml              False
1015252.female.23.indUnk.Pisces.xml               False
1015556.male.34.Technology.Virgo.xml              False
1016560.male.41.Publishing.Sagittarius.xml        False
1016738.male.26.Publishing.Libra.xml              False
1016787.female.24.Communications-Media.Leo.xml    False
1019224.female.27.RealEstate.Libra.xml          